In [1]:
#####有価証券報告書のスクレイピング#####
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from sqlalchemy import create_engine
from urllib.request import urlopen
import pandas as pd
import numpy as np
import os
import time
import itertools
import requests
import re

In [ ]:
##企業データの読み込みとクレンジング
#データの読み込み
company_info1 = pd.read_csv("D:/Statistics/data/stock_price/st-jp/company_info.csv")
index = np.where(np.in1d(company_info1["業種"], np.array(["ETF", "ETN", "REIT", "カントリーF", "インフラF"]))==False)[0]
company_info1 = company_info1.iloc[index]
company_info1.index = np.arange(company_info1.shape[0])
company_info2 = pd.read_csv("D:/Statistics/data/finance_data/stocklist.csv")
company_info2 = company_info2.rename(columns={"銘柄コード": "コード", "市場名": "取引市場", "業種分類": "業種"})
index = np.where(np.in1d(company_info2["業種"], np.array(["REIT銘柄一覧"]))==False)[0]
company_info2 = company_info2.iloc[index]
company_info2.index = np.arange(company_info2.shape[0])

#データの結合
company_info1 = company_info1[["コード", "銘柄名", "取引市場", "業種"]]
company_info2 = company_info2[["コード", "銘柄名", "取引市場", "業種"]]
company_info = pd.concat((company_info1, company_info2), axis=0)
company_info.index = np.arange(company_info.shape[0])

#変数を共通化
company = company_info[["コード", "銘柄名"]].iloc[np.where(company_info[["コード"]].duplicated()==False)[0]]
company_info["銘柄名"] = pd.merge(company_info[["コード"]], company, on="コード", how="left")["銘柄名"]
industry = np.array([['水産・農林業', '卸売業', '建設業', '非鉄金属', '鉱業', '機械', 'サービス業', '金属製品',
                      '情報・通信', '食料品', '医薬品', '不動産業', '陸運業', 'その他金融業', '小売業', 'その他製品',
                      '繊維製品', '電気機器', 'ガラス・土石製品', '証券業', '輸送用機器', '石油・石炭製品', '化学',
                      'パルプ・紙', '精密機器', 'ゴム製品', '鉄鋼', '銀行業', '保険業', '倉庫・運輸関連業', '海運業',
                      '空運業', '電気・ガス業'],
                   ['水産・農林', '卸売', '建設業', '非鉄金属', '鉱業', '機械', 'サービス', '金属製品',
                    '情報通信', '食料品', '医薬品', '不動産', '陸運', 'その他金融', '小売', 'その他製品', 
                    '繊維製品', '電気機器', 'ガラス土石', '証券・先物', '輸送用機器', '石油・石炭', '化学',
                    'パルプ・紙', '精密機器', 'ゴム製品', '鉄鋼', '銀行', '保険', '倉庫・運輸', '海運', 
                    '空運', '電気・ガス']])
industry_vec = np.array(company_info["業種"])
for j in range(industry.shape[1]):
    index = np.where(industry_vec==industry[0, j])[0]
    industry_vec[index] = np.array(industry[1, j], dtype="object")
company_info["業種"] = industry_vec
market = np.array([['東証1部', 'JQS', 'JQG', 'マザーズ', '東証2部', '福証Q', '名証2部', '札証', '福証',
                    '名証1部', '札証A', '名証C', '東証'],
                   ['東証1部', '東証JQS', '東証JQG', 'マザーズ', '東証2部', '福岡Q', '名証2部', '札証', "福証",
                    '名証1部',  '札幌ア', '名古屋セ', '東証']])
market_vec = np.array(company_info["取引市場"])
for j in range(market.shape[1]):
    index = np.where(market_vec==market[0, j])[0]
    market_vec[index] = np.array(market[1, j], dtype="object")
company_info["取引市場"] = market_vec

#重複を削除
company_info = company_info.iloc[np.where(company_info["コード"].duplicated()==False)[0]]
company_info.index = np.arange(company_info.shape[0])

In [ ]:
##有価証券報告書を取得するための情報をスクレイピング
#待ち時間を設定
second = np.arange(1, 5)
prob = np.repeat(1.0/second.shape[0], second.shape[0])

#データの格納用配列
code = np.array(company_info["コード"])
n = company_info.shape[0]
ticker_list = [i for i in range(n)]
company_list = [i for i in range(n)]
info_list = [i for i in range(n)]

In [ ]:
#すべての企業の有価証券報告書を取得するための情報を取得
for i in range(n):
    print([i, company_info["銘柄名"].iloc[i]])
    
    #URLを取得
    url = "http://www.kabupro.jp/yuho/{code}.htm"
    r = requests.get(url.format(code=code[i]))

    #短信のリンクを取得
    r.encoding = r.apparent_encoding
    html_text = r.text
    get_data = re.findall("[有価証券報告書|四半期報告書].+a href=.+日", html_text)

    #テキストを分解して必要情報を取得
    m = len(get_data)
    report = np.array(np.repeat("", m), dtype="object")
    season = np.array(np.repeat("", m), dtype="object")
    start_period = np.array(np.repeat("", m), dtype="object")
    end_period = np.array(np.repeat("", m), dtype="object")
    url = np.array(np.repeat("", m), dtype="object")
    for j in range(m):
        report0 = re.search(".+報告書", get_data[j])
        season0 = re.search("第.+期", get_data[j])
        period0= re.search("平成.+", get_data[j])
        url0 = re.search("a href=.+htm", get_data[j])
        if report0 is not None:
            report[j] = report0[0]
        if season0 is not None:
            season[j] = season0[0]
        if period0 is not None:
            split_period = re.split("-", period0[0])
            start_period[j] = split_period[0]
            end_period[j] = split_period[1]
        if url0 is not None:
            url[j] = "http://www.kabupro.jp" + url0[0][9:]
    ticker0 = np.repeat(code[i], m)
    company0 = np.array(company_info[["銘柄名", "取引市場", "業種"]].iloc[np.repeat(i, m)])
    info0 = np.vstack((report, season, start_period, end_period, url)).T

    #リストに格納
    ticker_list[i] = ticker0
    company_list[i] = company0
    info_list[i] = info0
    
    #待ち時間
    weight_time = np.dot(np.random.multinomial(1, prob, 1), second)
    time.sleep(weight_time)

In [ ]:
#リストをデータフレームに変換
ticker = np.array(list(itertools.chain(*[ticker_list[i] for i in range(m)])))
company = np.array(list(itertools.chain(*[company_list[i] for i in range(m)])))
info = np.array(list(itertools.chain(*[info_list[i] for i in range(m)])))
ticker_df = pd.DataFrame({"ticker": ticker})
company_df = pd.DataFrame(company)
company_df.columns = ["company", "market", "industry"]
info_df = pd.DataFrame(info)
info_df.columns = ["report", "season", "start_jp", "end_jp", "url"]
df = pd.concat((ticker_df, company_df, info_df), axis=1)

In [ ]:
##和暦を西暦に変換
#カレンダーの読み込み
N = df.shape[0]
calender = pd.read_csv("C:/statistics/data/finance_data/jpn_calender.csv")
finance_date = pd.read_csv("C:/statistics/data/finance_data/finance_date_list.csv")

In [ ]:
#日付を分割
start_date = np.zeros((N, 3), dtype="int16")
end_date = np.zeros((N, 3), dtype="int16")
start_period = np.repeat(0, N)
end_period = np.repeat(0, N)
temp1 = df["start_jp"].str[2:].str.split("[年|月|日]")
temp2 = df["end_jp"].str[2:].str.split("[年|月|日]")

#和暦を西暦に変換
for i in range(N):
    index1 = np.where(calender["jpn_calender"]==np.array(temp1[i][0], dtype="int"))[0]
    index2 = np.where(calender["jpn_calender"]==np.array(temp2[i][0], dtype="int"))[0]
    start_date[i, 0] = np.array(calender["calender"].iloc[index1], dtype="int16")
    end_date[i, 0] = np.array(calender["calender"].iloc[index2], dtype="int16")
    start_date[i, 1] = np.array(temp1[i][1], dtype="int16")
    end_date[i, 1] = np.array(temp2[i][1], dtype="int16")
    start_date[i, 2] = np.array(temp1[i][2], dtype="int16")
    end_date[i, 2] = np.array(temp2[i][2], dtype="int16")
    if len(temp1[i][1])==1:
        temp1[i][1] = "0" + temp1[i][1]
    if len(temp1[i][2])==1:
        temp1[i][2] = "0" + temp1[i][2]
    if len(temp2[i][1])==1:
        temp2[i][1] = "0" + temp2[i][1]
    if len(temp2[i][2])==1:
        temp2[i][2] = "0" + temp2[i][2]
    start_period[i] = np.array(np.array(start_date[i, 0], dtype="U"), dtype="object") + temp1[i][1] + temp1[i][2]
    end_period[i] = np.array(np.array(end_date[i, 0], dtype="U"), dtype="object") + temp2[i][1] + temp2[i][2]

In [ ]:
#データフレームに格納
start_df = pd.concat((pd.DataFrame(start_period), pd.DataFrame(start_date)), axis=1)
end_df = pd.concat((pd.DataFrame(end_period), pd.DataFrame(end_date)), axis=1)
start_df.columns = ["start_period", "start_year", "start_month", "start_day"]
end_df.columns = ["end_period", "end_year", "end_month", "end_day"]
target_df = pd.concat((df, start_df, end_df), axis=1)
target_df = target_df[["ticker", "company", "market", "industry", "report", "season", "start_period", "start_jp", "start_year", 
                       "start_month", "start_day", "end_period", "end_jp", "end_year", "end_month", "end_day", "url"]]
report_type = target_df["report"].str.replace("四半期", "")
end_period = target_df["end_period"].astype("U")
target_df["download_url"] = target_df["url"].str.replace("mark", "edp").str.replace("htm", "pdf")

#保存ファイルネームを設定
published = np.array(np.repeat("", target_df.shape[0]), dtype="object")
temp = target_df["download_url"].str.split("/")
for i in range(target_df.shape[0]):
    published[i] = temp[i][4] + temp[i][5].replace(".pdf", "")
target_df["filename"] = "【" + published + "】" + target_df["company"] + "_" + target_df["season"] + report_type + ".pdf"

In [2]:
target_df = pd.read_csv("D:/Statistics/data/finance_report/report_info.csv")
n = pd.unique(target_df["ticker"]).shape[0]

In [3]:
##有価証券報告書を取得
#待ち時間を設定
second = np.arange(1, 3)
prob = np.repeat(1.0/second.shape[0], second.shape[0])

#ディレクトリを作成
index_dup = np.array(np.where(target_df["ticker"].duplicated()==False)[0], dtype="int")
dir_name = np.array(target_df["ticker"].iloc[index_dup].astype("U") + "_" + target_df["company"].iloc[index_dup])
make_path = "D:/Statistics/data/finance_report/{name}"
for j in range(dir_name.shape[0]):
    create_name = make_path.format(name=dir_name[j])
    if os.path.exists(create_name)==False:
        os.mkdir(create_name)
    
#取得する企業と保存パスを設定
unique_ticker = pd.unique(target_df["ticker"])
n = unique_ticker.shape[0]
path = "D:/Statistics/data/finance_report/{dir}/{title}"

#企業毎に短信を取得してローカルに保存
for i in range(2414)[::-1]:
    
    #企業毎の取得urlを定義
    index = np.where(target_df["ticker"]==unique_ticker[i])[0]
    target_url = np.array(target_df["download_url"].iloc[index])
    m = index.shape[0]
    print([i, target_df["company"].iloc[index[0]]])

    #短信pdfを取得
    for j in range(m):
        url = target_url[j]
        req = requests.get(url)

        #pdfを保存
        target_name = np.array(target_df["filename"].iloc[index])
        target_path = path.format(dir=dir_name[i], title=target_name[j])
        myfile = open(target_path, "wb")
        myfile.write(req.content)
        
        #待ち時間
        time.sleep(1)

[2413, '日本ケミコン']
[2412, 'ニチコン']
[2411, '東海理化']
[2410, '指月電機製作所']
[2409, '大黒屋HD']
[2408, '北陸電気工業']
[2407, '日東電工']
[2406, '双葉電子工業']
[2405, 'ユーシン']
[2404, 'リード']
[2403, '村田製作所']
[2402, '日本抵抗器製作所']
[2401, '太陽誘電']
[2400, '協栄産業']
[2399, 'エルナー']
[2398, '京セラ']
[2397, '松尾電機']
[2396, '新光電気工業']
[2395, '三井ハイテック']
[2394, '浜松ホトニクス']
[2393, 'サンコー']
[2392, 'ローム']
[2391, '大真空']
[2390, 'エンプラス']
[2389, 'フクダ電子']
[2388, '日本ＣＭＫ']
[2387, '芝浦電子']
[2386, 'ＦＤＫ']
[2385, 'ファナック']
[2384, 'カシオ計算機']
[2383, '日本電子']
[2382, '図研']
[2381, '日本アビオニクス']
[2380, '富士通フロンテック']
[2379, 'アイレックス']
[2378, 'ＮＫＫスイッチズ']
[2377, 'ソフィアHD']
[2376, '山一電機']
[2375, '双信電機']
[2374, '古河電池']
[2373, '遠藤照明']
[2372, '日本アンテナ']
[2371, '日本セラミック']
[2370, 'エノモト']
[2369, 'ヘリオス_テクノ_HD']
[2368, '岡谷電機産業']
[2367, 'ウシオ電機']
[2366, '岩崎電気']
[2365, 'スタンレー電気']
[2364, 'レーザーテック']
[2363, 'ケル']
[2362, 'アバールデータ']
[2361, 'アイ・オー・データ機器']
[2360, '千代田インテグレ']
[2359, 'オプテックスグループ']
[2358, '菊水電子工業']
[2357, '新日本無線']
[2356, 'イリソ電子工業']
[2355, 'ジオマテック']
[2354, 'コーセル']
[2353, '原田工業']

KeyboardInterrupt: 

In [ ]:
2019-07-12 2282まで終了

In [ ]:
target_df.iloc[np.where(target_df["filename"].iloc[index].duplicated())[0]]